# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
vacation_data=pd.read_csv("../output_data/cities.csv")
vacation_df=pd.DataFrame(vacation_data)
vacation_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Ushuaia,40,AR,1597095386,80,-54.80,-68.30,33.80,18.34
1,Fairbanks,1,US,1597095327,58,64.84,-147.72,59.00,10.29
2,Xingtai,54,CN,1597095386,93,37.06,114.49,77.67,0.94
3,Oodweyne,61,SO,1597095386,66,9.41,45.06,73.81,13.33
4,Vanimo,71,PG,1597095386,83,-2.67,141.30,78.67,4.21
...,...,...,...,...,...,...,...,...,...
554,Faya,51,SA,1597095451,77,18.39,42.45,69.80,4.70
555,Emerald,0,AU,1597095451,81,-23.53,148.17,46.40,7.43
556,Canterbury,87,GB,1597095451,85,51.28,1.08,72.00,3.33
557,Ilebo,34,CD,1597095451,59,-4.32,20.58,77.86,0.45


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key = g_key)

locations = vacation_df[["Latitude", "Longitude"]]
humidity = vacation_df["Humidity"].astype(float)
max_humidity = humidity.max()

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=5)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
city_weather=vacation_df.loc[(vacation_df["Max Temp"]>70) & (vacation_df["Max Temp"]<80) & 
                             (vacation_df["Wind Speed"]<10) & (vacation_df["Cloudiness"]==0)].dropna()
city_weather

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
241,Tiznit Province,0,MA,1597095412,73,29.58,-9.50,71.51,2.21
259,Macatuba,0,BR,1597095414,37,-22.50,-48.71,73.99,1.01
312,Bengbu,0,CN,1597095419,95,32.94,117.36,74.68,4.47
354,Quirinópolis,0,BR,1597095424,35,-18.45,-50.45,74.84,1.03
516,Santa Cruz,0,MA,1597095378,78,30.42,-9.60,71.60,8.05
550,Itakyry,0,PY,1597095450,44,-24.93,-55.22,73.58,4.85


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = city_weather.loc[:,["City","Country", "Latitude", "Longitude"]]


hotel_df["Hotel Name"] = ""


hotel_df

,City,Country,Latitude,Longitude,Hotel Name
241,Tiznit Province,MA,29.58,-9.50,
259,Macatuba,BR,-22.50,-48.71,
312,Bengbu,CN,32.94,117.36,
354,Quirinópolis,BR,-18.45,-50.45,
516,Santa Cruz,MA,30.42,-9.60,
550,Itakyry,PY,-24.93,-55.22,


In [14]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": "5000",
    "type": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
# for i, row in perfect_cities_df.iterrows():
#     # get the latitude and longitude values
#     lat = row["Lat"]
#     lng = row["Lng"]
#     City = row["City"]
#     # add location to params dict
#     params["location"] = (f"{lat},{lng}")
#     # assemble url and make API request
#     print(f"Retrieving Results for City {City}.")
#     response = requests.get(base_url, params=params).json()
#     Printing the response so we can visualy locate the first hotel
#     print(json.dumps(response, indent=4, sort_keys=True))

In [ ]:
for index, row in hotel_df.iterrows():
    # getting city name, latitude, longitude from dataframe
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # adding keyword to params dictionary
    params["location"] = f"{lat},{lng}"

    # assembling the url and making API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # gathering the results
    results = response['results']
    
    # saving the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1.01)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 241: Tiznit Province.
Closest hotel in Tiznit Province is Ouijjane.
------------
Retrieving Results for Index 259: Macatuba.


In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig
